# Setup Cloud SQL - Python Connector
---

Install [cloud-sql-python connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector#how-to-use-this-connector) using [pymysql](https://github.com/PyMySQL/PyMySQL) as the driver

In [ ]:
!pip install cloud-sql-python-connector[pymysql]

Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

### Code

In [ ]:
# import connector
from google.cloud.sql.connector import connector, IPTypes

# import pandas
import pandas as pd

In [ ]:
conn = connector.connect(
    "dummy-329203:us-central1:dummy-mysql",
    "pymysql",
    ip_types=IPTypes.PUBLIC,
    user="root",
    password="CKnEx4mOtx2EADil",
    db="mysql"
)

In [ ]:
cursor = conn.cursor()
cursor.execute("SHOW COLUMNS FROM sys.sys_config")

columns = cursor.fetchall()
column_names = [col[0] for col in columns]

cursor.execute("SELECT * from sys.sys_config")
data = cursor.fetchall()

df = pd.DataFrame(data, columns=column_names)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None


# Setup Cloud SQL - SQL Alchemy
---

Install [pymysql](https://docs.sqlalchemy.org/en/14/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql)

In [6]:
!pip install pymysql

     |████████████████████████████████| 43 kB 2.0 MB/s 


Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

Need to add new network if use public IP: `0.0.0.0/0`



### Code

In [3]:
# import create_engine from sql alchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# import pandas
import pandas as pd

# import drive
from google.colab import drive

Mount gdrive folder to get SSL files

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


Build connection without SSL

In [ ]:
engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="dummy_db"
  )
)

conn = engine.connect()

Build connection with SSL

In [7]:
path_to_ssl = '/content/drive/MyDrive/Colab Notebooks/creds/SSL'

ssl_files={
    "ssl_ca": f"{path_to_ssl}/server-ca.pem",
    "ssl_cert": f"{path_to_ssl}/client-cert.pem",
    "ssl_key": f"{path_to_ssl}/client-key.pem"
}

engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="dummy_db"
  ),
  connect_args=ssl_files
)

conn = engine.connect()

Execute SQL query

Fetch methods:


1.   `.fetchall()`: to return all rows of query result
2.   `.fetchmany(size)`: to return number of rows specified by **size**
3.   `.fetchone()`: to return only the 1st row of query result



In [ ]:
query_get_cols = "SHOW COLUMNS FROM sys.sys_config"
query_result = conn.execute(query_get_cols)
column_names = query_result.fetchall()
columns = [col[0] for col in column_names]

query_get_data = "select * from sys.sys_config"
query_result = conn.execute(query_get_data)
data = query_result.fetchall()

df = pd.DataFrame(data, columns=columns)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None


# MySQL DB Commands
---

To list all databases / schemas

In [8]:
query = "SHOW DATABASES"
# query = "SHOW SCHEMAS"

To create database if not exists *(this query doesn't return a result)*

In [ ]:
query = """
CREATE DATABASE IF NOT EXISTS dummy_db;
"""

To delete database

In [10]:
query = "DROP DATABASE dummy_db"

To change database *(this query doesn't return a result)*

In [14]:
query = "USE dummy_db"

To see the active db

In [12]:
query = "SELECT DATABASE()"

To list all tables with the table type (table or view)

In [ ]:
query = "SHOW FULL TABLES"
# query = "SHOW FULL TABLES FROM mysql"

To check the table schema

In [ ]:
query = "describe dummy_db.post"

To execute the SQL query and return the result

In [13]:
query_result = conn.execute(query)
query_fetch_all = query_result.fetchall()

query_fetch_all

[('mysql',)]